In [80]:
using Zygote
using Distributions
using ITensors
using Random
using Plots 
using Base.Threads

In [81]:
struct PState
    pstate::MPS
    label::Int
end

In [82]:
function complex_feature_map(x::Float64)
    s1 = exp(1im * (3π/2) * x) * cospi(0.5 * x)
    s2 = exp(-1im * (2π/2) * x) * sinpi(0.5 * x)
    return [s1, s2]
end

complex_feature_map (generic function with 1 method)

In [83]:
function generate_training_data(samples_per_class::Int)

    class_A_samples = zeros(samples_per_class, 3)
    class_B_samples = ones(samples_per_class, 3)
    all_samples = vcat(class_A_samples, class_B_samples)
    all_labels = Int.(vcat(zeros(size(class_A_samples)[1]), ones(size(class_B_samples)[1])))

    return all_samples, all_labels

end

generate_training_data (generic function with 1 method)

In [84]:
function sample_to_product_state(sample::Vector, site_inds::Vector{Index{Int64}})
    n_sites = length(site_inds)
    product_state = MPS(ComplexF64, site_inds; linkdims=1)
    for j=1:n_sites
        T = ITensor(site_inds[j])
        zero_state, one_state = complex_feature_map(sample[j])
        T[1] = zero_state
        T[2] = one_state
        product_state[j] = T 
    end
    return product_state
end

sample_to_product_state (generic function with 1 method)

In [85]:
function dataset_to_product_state(dataset::Matrix, labels::Vector, sites::Vector{Index{Int64}})

    all_product_states = Vector{PState}(undef, size(dataset)[1])
    for p=1:length(all_product_states)
        sample_pstate = sample_to_product_state(dataset[p, :], sites)
        sample_label = labels[p]
        product_state = PState(sample_pstate, sample_label)
        all_product_states[p] = product_state
    end

    return all_product_states

end

dataset_to_product_state (generic function with 1 method)

In [86]:
s = siteinds("S=1/2", 3)
mps = randomMPS(ComplexF64, s; linkdims=4)
all_samples, all_labels = generate_training_data(100)
all_pstates = dataset_to_product_state(all_samples, all_labels, s);

In [87]:
mps

MPS
[1] ((dim=2|id=943|"S=1/2,Site,n=1"), (dim=4|id=25|"Link,l=1"))
[2] ((dim=4|id=25|"Link,l=1"), (dim=2|id=484|"S=1/2,Site,n=2"), (dim=2|id=880|"Link,l=2"))
[3] ((dim=2|id=880|"Link,l=2"), (dim=2|id=316|"S=1/2,Site,n=3"))


Make loss function. Takes in:
- Bond Tensor
- product state
- LE
- RE 
\
Outputs:
- Loss

In [88]:
N_train = 200
num_sites = 3

3

In [89]:
ITensor()

ITensor ord=0
NDTensors.EmptyStorage{NDTensors.EmptyNumber, NDTensors.Dense{NDTensors.EmptyNumber, Vector{NDTensors.EmptyNumber}}}

In [90]:
LE = Matrix{ITensor}(undef, N_train, num_sites);
RE = Matrix{ITensor}(undef, N_train, num_sites);

In [91]:
for i = 1:N_train
    RE[i, num_sites] = conj(all_pstates[i].pstate[num_sites]) * mps[num_sites]
end

for j = (num_sites-1):-1:1
    for i = 1:N_train
        RE[i, j] = RE[i, j+1] * mps[j] * conj(all_pstates[i].pstate[j])
    end
end

In [92]:
BT = mps[1] * mps[2]

ITensor ord=3 (dim=2|id=943|"S=1/2,Site,n=1") (dim=2|id=484|"S=1/2,Site,n=2") (dim=2|id=880|"Link,l=2")
NDTensors.Dense{ComplexF64, Vector{ComplexF64}}

In [93]:
function loss_per_sample(BT::ITensor, ps::PState, RE::Matrix, psid::Int, lid::Int, rid::Int)
    y = ps.label
    yhat = BT * conj(ps.pstate[lid]) * conj(ps.pstate[rid])
    yhat *= RE[psid, rid+1]
    
    yhat = yhat[]

    diff_sq = (abs((yhat - y)))^2

    loss = 0.5 * diff_sq

    return loss
    
end


loss_per_sample (generic function with 1 method)

In [95]:
nab = gradient(loss_per_sample, BT, all_pstates[1], RE, 1, 1, 2)[1]

ITensor ord=3 (dim=2|id=880|"Link,l=2") (dim=2|id=484|"S=1/2,Site,n=2") (dim=2|id=943|"S=1/2,Site,n=1")
NDTensors.Dense{ComplexF64, Vector{ComplexF64}}

In [105]:
@show nab

nab = ITensor ord=3
Dim 1: (dim=2|id=880|"Link,l=2")
Dim 2: (dim=2|id=484|"S=1/2,Site,n=2")
Dim 3: (dim=2|id=943|"S=1/2,Site,n=1")
NDTensors.Dense{ComplexF64, Vector{ComplexF64}}
 2×2×2
[:, :, 1] =
 0.007835624488941928 + 0.35001139410642584im  0.0 + 0.0im
 -0.02352789190592701 - 0.16642213948453546im  0.0 + 0.0im

[:, :, 2] =
 0.0 + 0.0im  0.0 + 0.0im
 0.0 + 0.0im  0.0 + 0.0im


ITensor ord=3 (dim=2|id=880|"Link,l=2") (dim=2|id=484|"S=1/2,Site,n=2") (dim=2|id=943|"S=1/2,Site,n=1")
NDTensors.Dense{ComplexF64, Vector{ComplexF64}}

In [103]:
function loss_and_grad_analytical(BT::ITensor, ps::PState, pid::Int, LE::Matrix, RE::Matrix, lid::Int, rid::Int)
    """Analytical form of the gradient for a single product state"""
    y = ps.label
    num_sites = length(ps.pstate)
    phi_tilde = conj(ps.pstate[lid]) * conj(ps.pstate[rid]) # two sites corresp to bond tensor

    if lid == 1
        # no left environment exists
        phi_tilde *= RE[pid, rid+1]
    elseif rid == num_sites
        phi_tilde *= LE[pid, lid-1]
    else
        phi_tilde *= LE[pid, lid-1] * RE[pid, rid+1]
    end

    # now for the loss
    yhat = BT * phi_tilde
    

    diff_sq = (abs(yhat[] - y))^2
    loss = 0.5 * diff_sq

    gradient = 0.5 * (yhat[]-y) * phi_tilde

    return loss, gradient

end

loss_and_grad_analytical (generic function with 3 methods)

In [104]:
loss, grad = loss_and_grad_analytical(BT, all_pstates[1], 1, LE, RE, 1, 2);
@show grad.tensor

grad.tensor = ComplexF64[0.11683788458079092 - 0.13035049666399298im 0.0 + 0.0im; 0.0 + 0.0im 0.0 + 0.0im;;; -0.06307257102486938 + 0.055536684599907474im 0.0 + 0.0im; 0.0 + 0.0im 0.0 + 0.0im]


Dim 1: (dim=2|id=943|"S=1/2,Site,n=1")
Dim 2: (dim=2|id=484|"S=1/2,Site,n=2")
Dim 3: (dim=2|id=880|"Link,l=2")
NDTensors.Dense{ComplexF64, Vector{ComplexF64}}
 2×2×2
[:, :, 1] =
 0.11683788458079092 - 0.13035049666399298im  0.0 + 0.0im
                 0.0 + 0.0im                  0.0 + 0.0im

[:, :, 2] =
 -0.06307257102486938 + 0.055536684599907474im  0.0 + 0.0im
                  0.0 + 0.0im                   0.0 + 0.0im

In [32]:
function loss_all(BT::ITensor, pss::Vector{PState}, RE::Matrix, lid::Int, rid::Int)
    # loop over all samples and get the loss
    losses = 0
    for (index, ps) in enumerate(pss)
        y = ps.label
        yhat = BT * conj(ps.pstate[lid]) * conj(ps.pstate[rid])
        yhat *= RE[index, rid+1]
        yhat = abs(yhat[])
        diff_sq = (yhat - y)^2
        loss = 0.5 * diff_sq
        losses += loss
    end

    total_loss = losses / length(pss)
    return total_loss

end

loss_all (generic function with 1 method)

In [33]:
loss_all(BT, all_pstates, RE, 1, 2)

0.0980056452410914

In [18]:
loss_per_sample(BT, all_pstates[101], RE, 101, 1, 2)

0.1664146844804412

Returns gradient of loss function with resepct to each argument, so it will return 6 outputs. We only want the first (w.r.t. BT)

In [42]:
using BenchmarkTools

In [50]:
@benchmark gradient(loss_all, BT, all_pstates, RE, 1, 2)

BenchmarkTools.Trial: 49 samples with 1 evaluation.
 Range (min … max):  101.130 ms … 116.523 ms  ┊ GC (min … max): 0.00% … 1.52%
 Time  (median):     103.181 ms               ┊ GC (median):    0.00%
 Time  (mean ± σ):   103.766 ms ±   2.438 ms  ┊ GC (mean ± σ):  0.92% ± 1.02%

      █▄     ▄  ▂▂                                               
  ▆▆▆████▁▄▄▄█▄▆██▆▁▁▁▁▄▁▄▁▁▁▁▄▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▄ ▁
  101 ms           Histogram: frequency by time          117 ms <

 Memory estimate: 20.25 MiB, allocs estimate: 253138.

In [36]:
f, (∇,) = withgradient(loss_per_sample, BT, all_pstates[101], RE, 101, 1, 2)

(val = 0.1664146844804412, grad = (ITensor ord=3
Dim 1: (dim=2|id=386|"Link,l=2")
Dim 2: (dim=2|id=777|"S=1/2,Site,n=2")
Dim 3: (dim=2|id=711|"S=1/2,Site,n=1")
NDTensors.Dense{ComplexF64, Vector{ComplexF64}}
 2×2×2
[:, :, 1] =
 0.0 + 0.0im  0.0 + 0.0im
 0.0 + 0.0im  0.0 + 0.0im

[:, :, 2] =
 0.0 + 0.0im   0.12236994843263635 - 0.5586193164948039im
 0.0 + 0.0im  0.037272833889576744 - 0.06640903382970201im, (pstate = (data = Union{Nothing, ITensor}[ITensor ord=1
Dim 1: (dim=2|id=711|"S=1/2,Site,n=1")
NDTensors.Dense{ComplexF64, Vector{ComplexF64}}
 2-element
 -0.2003500985052977 + 0.08898968182014054im
  0.2440842891898694 + 6.474164065252424e-17im, ITensor ord=1
Dim 1: (dim=2|id=777|"S=1/2,Site,n=2")
NDTensors.Dense{ComplexF64, Vector{ComplexF64}}
 2-element
 0.0017623652742804123 - 0.27960794579555726im
    0.4881685783797388 + 1.215852552788401e-16im, nothing], llim = nothing, rlim = nothing), label = 0.5769136581507517), Union{Nothing, ITensor}[nothing nothing nothing; nothing nothi

In [48]:
function test_alternative_approach()
    nabs = Vector{ITensor}(undef, 200)
    fs = Vector{Float64}(undef, 200)
    for i=1:200
        f, (∇,) = withgradient(loss_per_sample, BT, all_pstates[i], RE, i, 1, 2)
        nabs[i] = ∇
        fs[i] = f
    end
end

test_alternative_approach (generic function with 1 method)

In [51]:
@benchmark test_alternative_approach()

BenchmarkTools.Trial: 31 samples with 1 evaluation.
 Range (min … max):  159.902 ms … 170.203 ms  ┊ GC (min … max): 0.00% … 1.06%
 Time  (median):     162.480 ms               ┊ GC (median):    0.00%
 Time  (mean ± σ):   163.108 ms ±   2.504 ms  ┊ GC (mean ± σ):  0.45% ± 0.58%

  █ █▁▁▁ ▁ ▁▁█ █▁▁   ▁▁██ ▁ ▁ ▁ █         ▁  ▁    ▁           ▁  
  █▁████▁█▁███▁███▁▁▁████▁█▁█▁█▁█▁▁▁▁▁▁▁▁▁█▁▁█▁▁▁▁█▁▁▁▁▁▁▁▁▁▁▁█ ▁
  160 ms           Histogram: frequency by time          170 ms <

 Memory estimate: 17.73 MiB, allocs estimate: 203719.

In [37]:
nabs = Vector{ITensor}(undef, 200)
fs = Vector{Float64}(undef, 200)
for i=1:200
    f, (∇,) = withgradient(loss_per_sample, BT, all_pstates[i], RE, i, 1, 2)
    nabs[i] = ∇
    fs[i] = f
end

In [38]:
∇_total = sum(nabs)
fs_total = sum(fs)

19.601129048218283

In [39]:
gradient_tensor = ∇_total ./ 200
loss_final = fs_total / 200

0.09800564524109141

In [41]:
@show gradient_tensor

gradient_tensor = ITensor ord=3
Dim 1: (dim=2|id=386|"Link,l=2")
Dim 2: (dim=2|id=777|"S=1/2,Site,n=2")
Dim 3: (dim=2|id=711|"S=1/2,Site,n=1")
NDTensors.Dense{ComplexF64, Vector{ComplexF64}}
 2×2×2
[:, :, 1] =
 0.004857329134037524 - 0.0153055904724757im   0.0 + 0.0im
 -0.06839490082813432 + 0.09931055228702056im  0.0 + 0.0im

[:, :, 2] =
 0.0 + 0.0im  0.06118497421631818 - 0.2793096582474026im
 0.0 + 0.0im  0.01863641694478835 - 0.03320451691485096im


ITensor ord=3 (dim=2|id=386|"Link,l=2") (dim=2|id=777|"S=1/2,Site,n=2") (dim=2|id=711|"S=1/2,Site,n=1")
NDTensors.Dense{ComplexF64, Vector{ComplexF64}}

Apply update

In [23]:
BT_old = BT

ITensor ord=3 (dim=2|id=711|"S=1/2,Site,n=1") (dim=2|id=777|"S=1/2,Site,n=2") (dim=2|id=386|"Link,l=2")
NDTensors.Dense{ComplexF64, Vector{ComplexF64}}

In [24]:
lr = 0.5
BT_new = BT_old - lr * gradient_tensor

ITensor ord=3 (dim=2|id=711|"S=1/2,Site,n=1") (dim=2|id=777|"S=1/2,Site,n=2") (dim=2|id=386|"Link,l=2")
NDTensors.Dense{ComplexF64, Vector{ComplexF64}}

In [25]:
loss_per_sample(BT_new, all_pstates[101], RE, 101, 1, 2)

0.093608260020248

#### SVD apart new bond tensor

In [349]:
BT_new

ITensor ord=3 (dim=2|id=703|"S=1/2,Site,n=1") (dim=2|id=922|"S=1/2,Site,n=2") (dim=2|id=767|"Link,l=2")
NDTensors.Dense{ComplexF64, Vector{ComplexF64}}

In [350]:
left_site_index = uniqueinds(mps[1], mps[2])

1-element Vector{Index{Int64}}:
 (dim=2|id=703|"S=1/2,Site,n=1")

In [351]:
U, S, V = svd(BT_new, left_site_index; lefttags="Link,l=1");

In [352]:
U

ITensor ord=2 (dim=2|id=703|"S=1/2,Site,n=1") (dim=2|id=986|"Link,l=1")
NDTensors.Dense{ComplexF64, Vector{ComplexF64}}

In [353]:
S

ITensor ord=2 (dim=2|id=986|"Link,l=1") (dim=2|id=211|"Link,v")
NDTensors.Diag{Float64, Vector{Float64}}

In [354]:
V

ITensor ord=3 (dim=2|id=922|"S=1/2,Site,n=2") (dim=2|id=767|"Link,l=2") (dim=2|id=211|"Link,v")
NDTensors.Dense{ComplexF64, Vector{ComplexF64}}

In [355]:
left_site_new = U
right_site_new = S * V

ITensor ord=3 (dim=2|id=986|"Link,l=1") (dim=2|id=922|"S=1/2,Site,n=2") (dim=2|id=767|"Link,l=2")
NDTensors.Dense{ComplexF64, Vector{ComplexF64}}

In [356]:
for i = 1:200
    LE[i, 1] = left_site_new * conj(all_pstates[i].pstate[1])
end

Add sites back into the mps

In [357]:
mps[1] = left_site_new

ITensor ord=2 (dim=2|id=703|"S=1/2,Site,n=1") (dim=2|id=986|"Link,l=1")
NDTensors.Dense{ComplexF64, Vector{ComplexF64}}

In [358]:
mps[2] = right_site_new

ITensor ord=3 (dim=2|id=986|"Link,l=1") (dim=2|id=922|"S=1/2,Site,n=2") (dim=2|id=767|"Link,l=2")
NDTensors.Dense{ComplexF64, Vector{ComplexF64}}

In [359]:
normalize!(mps)

MPS
[1] ((dim=2|id=703|"S=1/2,Site,n=1"), (dim=2|id=986|"Link,l=1"))
[2] ((dim=2|id=986|"Link,l=1"), (dim=2|id=922|"S=1/2,Site,n=2"), (dim=2|id=767|"Link,l=2"))
[3] ((dim=2|id=767|"Link,l=2"), (dim=2|id=503|"S=1/2,Site,n=3"))


# Now for sites 2-3

In [360]:
BT_s2s3 = mps[2] * mps[3]

ITensor ord=3 (dim=2|id=986|"Link,l=1") (dim=2|id=922|"S=1/2,Site,n=2") (dim=2|id=503|"S=1/2,Site,n=3")
NDTensors.Dense{ComplexF64, Vector{ComplexF64}}

In [380]:
function loss_per_sample2(BT::ITensor, ps::PState, LE::Matrix, psid::Int, lid::Int, rid::Int)
    y = ps.label
    yhat = BT * conj(ps.pstate[lid]) * conj(ps.pstate[rid])
    yhat *= LE[psid, lid-1]
    
    yhat = abs(yhat[])

    diff_sq = (yhat - y)^2

    loss = 0.5 * diff_sq

    return loss
    
end


loss_per_sample2 (generic function with 2 methods)

In [394]:
loss_per_sample2(BT_s2s3, all_pstates[101], LE, 101, 2, 3)

0.16517458495287118

In [375]:
LE[:, 2] = LE[:, 1]
LE[:, 3] = LE[:, 1]

200-element Vector{ITensor}:
 ITensor ord=1
Dim 1: (dim=2|id=986|"Link,l=1")
NDTensors.Dense{ComplexF64, Vector{ComplexF64}}
 2-element
 -0.7171921444508376 + 4.6419259161868125e-18im
 -0.6968754752019972 - 4.9483564363427824e-18im
 ITensor ord=1
Dim 1: (dim=2|id=986|"Link,l=1")
NDTensors.Dense{ComplexF64, Vector{ComplexF64}}
 2-element
 -0.7171921444508376 + 4.6419259161868125e-18im
 -0.6968754752019972 - 4.9483564363427824e-18im
 ITensor ord=1
Dim 1: (dim=2|id=986|"Link,l=1")
NDTensors.Dense{ComplexF64, Vector{ComplexF64}}
 2-element
 -0.7171921444508376 + 4.6419259161868125e-18im
 -0.6968754752019972 - 4.9483564363427824e-18im
 ITensor ord=1
Dim 1: (dim=2|id=986|"Link,l=1")
NDTensors.Dense{ComplexF64, Vector{ComplexF64}}
 2-element
 -0.7171921444508376 + 4.6419259161868125e-18im
 -0.6968754752019972 - 4.9483564363427824e-18im
 ITensor ord=1
Dim 1: (dim=2|id=986|"Link,l=1")
NDTensors.Dense{ComplexF64, Vector{ComplexF64}}
 2-element
 -0.7171921444508376 + 4.6419259161868125e-18im
 -0.

In [377]:
nab = gradient(loss_per_sample2, BT_s2s3, all_pstates[1], LE, 1, 2, 3)

(ITensor ord=3
Dim 1: (dim=2|id=986|"Link,l=1")
Dim 2: (dim=2|id=503|"S=1/2,Site,n=3")
Dim 3: (dim=2|id=922|"S=1/2,Site,n=2")
NDTensors.Dense{ComplexF64, Vector{ComplexF64}}
 2×2×2
[:, :, 1] =
 -0.049483289637400146 + 0.2229569378368305im   0.0 + 0.0im
  -0.04808152354628181 + 0.21664099810183296im  0.0 + 0.0im

[:, :, 2] =
 0.0 + 0.0im  0.0 + 0.0im
 0.0 + 0.0im  0.0 + 0.0im, (pstate = (data = Union{Nothing, ITensor}[nothing, ITensor ord=1
Dim 1: (dim=2|id=922|"S=1/2,Site,n=2")
NDTensors.Dense{ComplexF64, Vector{ComplexF64}}
 2-element
 0.10140354704800512 + 1.2042038752093967e-18im
 0.07096073107074409 + 0.05167708915573277im, ITensor ord=1
Dim 1: (dim=2|id=503|"S=1/2,Site,n=3")
NDTensors.Dense{ComplexF64, Vector{ComplexF64}}
 2-element
 0.20280709409601025 + 2.4084077504187934e-18im
 0.09035702443087401 - 0.04775258323717094im], llim = nothing, rlim = nothing), label = -0.3184392360372778), Union{Nothing, ITensor}[ITensor ord=1
Dim 1: (dim=2|id=986|"Link,l=1")
NDTensors.Dense{Complex

In [378]:
nab[1]

ITensor ord=3 (dim=2|id=986|"Link,l=1") (dim=2|id=503|"S=1/2,Site,n=3") (dim=2|id=922|"S=1/2,Site,n=2")
NDTensors.Dense{ComplexF64, Vector{ComplexF64}}

In [382]:
nabs = Vector{ITensor}(undef, 200)
fs = Vector{Float64}(undef, 200)
for i=1:200
    f, (∇,) = withgradient(loss_per_sample2, BT_s2s3, all_pstates[i], LE, i, 2, 3)
    nabs[i] = ∇
    fs[i] = f
end

In [385]:
∇_total = sum(nabs)
fs_total = sum(fs)

21.58763584768739

In [386]:
gradient_tensor = ∇_total ./ 200
loss_final = fs_total / 200

0.10793817923843696

In [387]:
BT_old = BT_s2s3

ITensor ord=3 (dim=2|id=986|"Link,l=1") (dim=2|id=922|"S=1/2,Site,n=2") (dim=2|id=503|"S=1/2,Site,n=3")
NDTensors.Dense{ComplexF64, Vector{ComplexF64}}

In [388]:
lr = 0.5
BT_new = BT_old - lr * gradient_tensor

ITensor ord=3 (dim=2|id=986|"Link,l=1") (dim=2|id=922|"S=1/2,Site,n=2") (dim=2|id=503|"S=1/2,Site,n=3")
NDTensors.Dense{ComplexF64, Vector{ComplexF64}}

In [395]:
loss_per_sample2(BT_new, all_pstates[101], LE, 101, 2, 3)

0.09291070403599008

In [396]:
left_site_index = uniqueinds(mps[2], mps[3])

2-element Vector{Index{Int64}}:
 (dim=2|id=986|"Link,l=1")
 (dim=2|id=922|"S=1/2,Site,n=2")

In [397]:
U, S, V = svd(BT_new, left_site_index; lefttags="Link,l=2")

ITensors.TruncSVD(ITensor ord=3
Dim 1: (dim=2|id=986|"Link,l=1")
Dim 2: (dim=2|id=922|"S=1/2,Site,n=2")
Dim 3: (dim=2|id=20|"Link,l=2")
NDTensors.Dense{ComplexF64, Vector{ComplexF64}}
 2×2×2
[:, :, 1] =
 -0.3455968705625582 + 0.08904580273074056im  …    0.7960877894942596 - 0.26786833948040506im
   0.199377085293357 - 0.29330252798591516im     -0.07330340522004175 - 0.18966667581760785im

[:, :, 2] =
 -0.061426409141553825 - 0.48859884184919017im  …  -0.32048491126437545 - 0.24380718428862477im
    0.6507872317632465 - 0.3843265599983129im      0.014211587966985833 + 0.15463683767451142im, ITensor ord=2
Dim 1: (dim=2|id=20|"Link,l=2")
Dim 2: (dim=2|id=677|"Link,v")
NDTensors.Diag{Float64, Vector{Float64}}
 2×2
 1.0149924244704531  0.0
 0.0                 0.3022111408935106, ITensor ord=2
Dim 1: (dim=2|id=503|"S=1/2,Site,n=3")
Dim 2: (dim=2|id=677|"Link,v")
NDTensors.Dense{ComplexF64, Vector{ComplexF64}}
 2×2
 -0.5007863554939016 + 0.0im                 …  -0.8655709249686796 + 0.0im
 

In [403]:
left_site_new = U
right_site_new = S * V

ITensor ord=2 (dim=2|id=20|"Link,l=2") (dim=2|id=503|"S=1/2,Site,n=3")
NDTensors.Dense{ComplexF64, Vector{ComplexF64}}

In [411]:
mps[2] = left_site_new
mps[3] = right_site_new

ITensor ord=2 (dim=2|id=20|"Link,l=2") (dim=2|id=503|"S=1/2,Site,n=3")
NDTensors.Dense{ComplexF64, Vector{ComplexF64}}

In [413]:
normalize!(mps)

MPS
[1] ((dim=2|id=703|"S=1/2,Site,n=1"), (dim=2|id=986|"Link,l=1"))
[2] ((dim=2|id=986|"Link,l=1"), (dim=2|id=922|"S=1/2,Site,n=2"), (dim=2|id=20|"Link,l=2"))
[3] ((dim=2|id=20|"Link,l=2"), (dim=2|id=503|"S=1/2,Site,n=3"))


In [422]:
res = 1
for i=1:3
    res *= mps[i] * all_pstates[101].pstate[i]
end